**Engy-5310** Computational Continuum Transport Phenomena, UMass Lowell, Chem. Eng. Nuclear Program Spring 2021; Prof. V. F. de Almeida

# Xenon-135 Removal from Molten Salt Reactors by Helium Gas Bubbles
$  
  \newcommand{\Amtrx}{\boldsymbol{\mathsf{A}}}
  \newcommand{\Bmtrx}{\boldsymbol{\mathsf{B}}}
  \newcommand{\Mmtrx}{\boldsymbol{\mathsf{M}}}
  \newcommand{\Imtrx}{\boldsymbol{\mathsf{I}}}
  \newcommand{\Pmtrx}{\boldsymbol{\mathsf{P}}}
  \newcommand{\Lmtrx}{\boldsymbol{\mathsf{L}}}
  \newcommand{\Umtrx}{\boldsymbol{\mathsf{U}}}
  \newcommand{\Smtrx}{\boldsymbol{\mathsf{S}}}
  \newcommand{\xvec}{\boldsymbol{\mathsf{x}}}
  \newcommand{\uvar}{\boldsymbol{u}}
  \newcommand{\fvar}{\boldsymbol{f}}
  \newcommand{\avec}{\boldsymbol{\mathsf{a}}}
  \newcommand{\bvec}{\boldsymbol{\mathsf{b}}}
  \newcommand{\cvec}{\boldsymbol{\mathsf{c}}}
  \newcommand{\rvec}{\boldsymbol{\mathsf{r}}}
  \newcommand{\mvec}{\boldsymbol{\mathsf{m}}}
  \newcommand{\gvec}{\boldsymbol{\mathsf{g}}}
  \newcommand{\zerovec}{\boldsymbol{\mathsf{0}}}
  \newcommand{\norm}[1]{\bigl\lVert{#1}\bigr\rVert}
  \newcommand{\transpose}[1]{{#1}^\top}
  \DeclareMathOperator{\rank}{rank}
  \newcommand{\Power}{\mathcal{P}}
$

**Student(s):** Anadi Mondal, Anthony G. Bowers Jr. <br>
**Course:** Engy-5310 Computational Continuum Transport Phenomena  <br>
**Mentor:**Prof. Subash L. Sharma 

UMass Lowell, Dept. of Chemical Engineering, Nuclear Energy Program 
<br>
**27Mar2021**

---
## Table of Contents<a id="toc"></a>
* [Introduction](#intro)
* [Problem Statement](#problem)
* [Objectives](#obj)
* [Approach](#app)
 + [Weak Form of Problem Statement](#weakform)
 + [MOOSE Kernels](#kernels)
 + [Setup](#setup)
 + [Tests](#tests)
   - [1-D](#1d-test)
   - [2-D](#1d-test)
* [Results and Analysis](#ranalysis)
* [Conclusions](#conclusions)
* [References](#refs)
---

## [Introduction](#toc)<a id="intro"></a>

- Xenon-135 buildup in the molten salt reactor is dangerous for the sustaining running of reactor. As xenon has very high absorption cross section, it absorb neutron which hinders the continuous chain reaction for required power production. So, removing of xenon form the reactor core is must. One of the ways to remove xenon is to sparge helium gas as bubble in the reactor core because xenon is easily absorbed in helium bubble. The amount of xenon transfer to the helium is highly dependent on helium void fraction. If we increase the void fraction of helium in the system it will increase the interfacial area which is responsible for transferring xenon from salt to helium bubble.

## [Problem Statement](#toc)<a id="problem"></a>

- The Xe-135 is stripped from the Molten salt by interfacial Mass transfer process using the He Gas. The general transport equation for this is

So the first step is in this direction is to simplify the problem. 

- The strong form of the two coupled equations are written as,

The mass conservation equation of xenon in salt,



And the conservation equation of xenon in helium gas is as follows,




- Text... 

## [Objectives](#toc)<a id="obj"></a>

- To determine the concentration change of Xenon in fuel salt
- To determine the buildup of xenon in helium bubble


- Text...

## [Approach](#toc)<a id="app"></a>

- The weak form of the two conservation equation can be written as,

- Need to use 3 MOOSE kernels
1. Diffusion Kernel
2. Convection Kernel
3. Source Kernel

In [ ]:

'''Plot function for FEM Solution'''

def plot_solution(df, 
                  title='No Title', 
                  u1_legend='no u1 legend',
                  u2_legend='no u2 legend',
                  u1_flux_legend='no u1 flux legend',
                  u2_flux_legend='no u2 flux legend'):
    
    import matplotlib.pyplot as plt
    %matplotlib inline
    plt.style.use('dark_background')

    (fig, ax1) = plt.subplots(1, figsize=(15, 6))

    ax1.plot(df['x'], df['u'],'r*-',label=u1_legend)
    
    if 'u2' in df.columns:
        ax1.plot(df['x'], df['u2'],'r*--',label=u2_legend)

    ax1.set_xlabel(r'$x$ [cm]', fontsize=18)
    ax1.set_ylabel(r'$u_h(x)$ [g/cc]', fontsize=18, color='red')
    ax1.tick_params(axis='y', labelcolor='red', labelsize=14)
    ax1.tick_params(axis='x', labelsize=14)
    ax1.legend(loc='center left', fontsize=12)
    #ax1.set_ylim(0,1)
    ax1.grid(True)

    if 'diffFluxU_x' in df.columns:
        # create a twin x axis to be shared
        ax2 = ax1.twinx()

        ax2.plot(df['x'], df['diffFluxU_x'],'*-', color='yellow', label=u1_flux_legend)
        
        if 'diffFluxU2_x' in df.columns:
            ax2.plot(df['x'], df['diffFluxU2_x'],'*--', color='yellow', label=u2_flux_legend)

        ax2.set_ylabel(r"$q_h(x)$ [g/cm$^2$-s]", fontsize=16, color='yellow')
        ax2.tick_params(axis='y', labelcolor='yellow', labelsize=14)
        ax2.legend(loc='center right', fontsize=12)
        #ax2.set_ylim(0,2)
        #ax2.grid(True)

    plt.title(title, fontsize=20)
    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.show()
    print('')

In [ ]:
'''Domain'''

x_a = 0
x_b = 6

x_length = x_b - x_a

In [ ]:

'''Parameters and data'''

Pe_ave = 10 # mildly convective dominated

diff_coeff_1 = 0.1
source_s_1 = 1e-3
source_transfer_coeff_1 = 5e-3
source_saturation_1 = 1.0

diff_coeff_2 = diff_coeff_1

velocity = (Pe_ave * (diff_coeff_1+diff_coeff_2)/2/x_length, 0, 0)  # length scale is the x length

u_a = 3
u_b = 0

u2_a = 0
u2_b = 0

In [ ]:
'''FEM Solution'''

n_felem = 20

order = 'first'

n_plot_pts = n_felem + 1

try:    
    from engy_5310.toolkit import write_engy5310_p1_1d_input_file  
except ModuleNotFoundError:
    assert False, 'You need to provide your own code here. Bailing out.'

write_engy5310_p1_1d_input_file(x_left=x_a, x_right=x_b, 
                                u_left=u_a, u_right=u_b, 
                                diff_coeff=diff_coeff_1, source_s=source_s_1,
                                source_transfer_coeff=source_transfer_coeff_1, 
                                source_saturation=source_saturation_1,
                                u2_left=u2_a, u2_right=u2_b, 
                                diff_coeff_2=diff_coeff_2,
                                velocity=velocity, 
                                n_felem=n_felem, order=order, 
                                n_plot_pts=n_plot_pts,
                                compute_diffusion_flux=True,
                                solver='fdp-newt-full')

## [Results and Analysis](#toc)<a id="ranalysis"></a>

- The concentration change of xenon in salt and helium are as follows

In [ ]:

'''Show FEM Solution'''

import pandas as pd
df = pd.read_csv('output_x-data_0002.csv')
    
plot_solution(df, title='Peclet Coupled Variables w/ Dirichlet BC FEM Solution', 
              u1_legend=r'$u_1$ Linear Lagrange', u2_legend=r'$u_2$ Linear Lagrange',
              u1_flux_legend=r'$u_1$ Diff. Flux Constant Monomial',
              u2_flux_legend=r'$u_2$ Diff. Flux Constant Monomial')

## [Conclusions](#toc)<a id="conclusions"></a>

- The concentration of xenon in fuel salt is decreased due to transfer of xenon from salt to helium bubble. MOOSE and analytical solution gave the same result. To improve the result I need to consider several variable term in the governing equation. This is the simplest forms and considerations to simulate the concentration change of xenon in molten salt reactor.

## [References](#toc)<a id="refs"></a>

- [1] V. F. de Almeida, [*Engy-5310: Computational Continuum Transport Phenomena*](https://github.com/dpploy/engy-5310), University of Massachusetts Lowell, Dept. of Chemical Engineering (Nuclear Energy Program).
- [2] Multiphysics Object-Oriented Simulation Environment [(MOOSE)](https://mooseframework.org)
- [3] author, [reference]()
- [4] author, [reference]()